In [1]:
import tensorflow as tf
import symae_core as symae
tfk = tf.keras
tfkl = tf.keras.layers
tfkltd= tf.keras.layers.TimeDistributed
d=1
n1=100
n2=100
N=1000
nt=20
p=8
q=22
dropout_rate=0.5

2022-06-16 15:50:03.033697: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Num GPUs Available:  2
TensorFlow Version:  2.9.1


In [2]:
# Build symmetric encoder
sym_enc_input=tfk.Input(shape=(nt,1))
sym_enc_output=symae.SymmetricEncoderDense0D(p,nt)(sym_enc_input)
sym_encoder=tfk.Model(sym_enc_input, sym_enc_output, name='sym_encoder')
sym_encoder.summary()

2022-06-16 15:50:10.835520: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-16 15:50:12.224482: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 714 MB memory:  -> device: 0, name: NVIDIA RTX A4000, pci bus id: 0000:b1:00.0, compute capability: 8.6
2022-06-16 15:50:12.225389: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13931 MB memory:  -> device: 1, name: NVIDIA RTX A4000, pci bus id: 0000:b2:00.0, compute capability: 8.6


Model: "sym_encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20, 1)]           0         
                                                                 
  (SymmetricEncoderDense0D)  (None, 8)                 478       
                                                                 
Total params: 478
Trainable params: 458
Non-trainable params: 20
_________________________________________________________________


In [3]:
# Build nuisance encoder
nui_enc_input=tfk.Input(shape=(nt,1), dtype='float32', name='sym_enc_input')
nui_enc_output=symae.NuisanceEncoderDense0D(q,nt)(nui_enc_input)
nui_enc_flatten=tfkl.Flatten()(nui_enc_output)
nui_encoder=tfk.Model(nui_enc_input, nui_enc_flatten, name='nui_encoder')
nui_encoder.summary()

Model: "nui_encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sym_enc_input (InputLayer)  [(None, 20, 1)]           0         
                                                                 
  (NuisanceEncoderDense0D)   (None, 20, 22)            522       
                                                                 
 flatten (Flatten)           (None, 440)               0         
                                                                 
Total params: 522
Trainable params: 502
Non-trainable params: 20
_________________________________________________________________


In [4]:
# Build distribute in decoder
distribute_input = tfk.Input(shape=(p+q*nt), name='latentcode')
distribute_output=symae.DistributeZsym(nt, p, q)(distribute_input)
distzsym = tfk.Model(distribute_input, distribute_output, name='distzsym')
distzsym.summary()

Model: "distzsym"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 latentcode (InputLayer)     [(None, 448)]             0         
                                                                 
  (DistributeZsym)           (None, 20, 30)            0         
                                                                 
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [5]:
#Build mixer in decoder
mixer_input = tfk.Input(shape=(nt,p+q), name='mixer_input')
mixer_output=symae.MixerDense0D(1,p+q)(mixer_input)
mixer = tfk.Model(mixer_input, mixer_output, name='mixer') 
mixer.summary()

Model: "mixer"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mixer_input (InputLayer)    [(None, 20, 30)]          0         
                                                                 
  (MixerDense0D)             (None, 20, 1)             1021      
                                                                 
Total params: 1,021
Trainable params: 991
Non-trainable params: 30
_________________________________________________________________


In [6]:
# Build encoder
encoder_input=tfk.Input(shape=(nt,1), dtype='float32', name='encoder_input')
znuisance=nui_encoder(encoder_input)
zsym=sym_encoder(encoder_input)
latentcat=symae.LatentCat(dropout_rate) #0.4 droupout
encoder_output=latentcat(zsym,znuisance)
encoder=tfk.Model(encoder_input, encoder_output, name="encoder")

# Build decoder
decoder_input = tfk.Input(shape=(p+q*nt), name='latentcode')
decoder_output=mixer(distzsym(decoder_input))
decoder=tfk.Model(decoder_input,decoder_output, name="decoder") 

#Build SymAE
model=tf.keras.Model(encoder_input, decoder(encoder_output) , name='autoencoder_clone')

In [7]:
# Generate the Dataset
import numpy as np
d=1
n1=100
n2=100
nx=10
nt=20
p=8
q=22
states=np.arange(n1)
nuisances=np.arange(n2)
X=np.empty((nx,nt), dtype = float, order = 'C')
D=np.tensordot(states,nuisances,0)
for i in range(nx):
    e=np.random.choice(range(n1))
    c=np.random.choice(range(n2),replace=False,size=nt)
    for t in range(nt):
        X[i,t]=D[e,c[t]]    

In [8]:
Adam=tf.keras.optimizers.Adam(learning_rate=0.001,beta_1=0.9,beta_2=0.999,epsilon=1e-07)
SGD=tf.keras.optimizers.SGD(learning_rate=0.1,momentum=0.9,nesterov=True)
model.compile(loss='mse',optimizer=Adam)
model.summary()

Model: "autoencoder_clone"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input (InputLayer)     [(None, 20, 1)]      0           []                               
                                                                                                  
 sym_encoder (Functional)       (None, 8)            478         ['encoder_input[0][0]']          
                                                                                                  
 nui_encoder (Functional)       (None, 440)          522         ['encoder_input[0][0]']          
                                                                                                  
  (LatentCat)                   (None, 448)          0           ['sym_encoder[0][0]',            
                                                                  'nui_encoder[0][

In [9]:
# Train the NeuralNet
M=50
epochs=range(M)
losses=[0.0]*M
class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        clear_output(wait=True)
        losses[epoch]=logs["loss"]
        print("For epoch {:d}, loss is {:f}.".format(epoch, logs["loss"]))
history=model.fit(X,X,epochs=M,verbose=0,callbacks=[CustomCallback()])

2022-06-16 15:50:16.242676: E tensorflow/stream_executor/cuda/cuda_blas.cc:232] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2022-06-16 15:50:16.242724: E tensorflow/stream_executor/cuda/cuda_blas.cc:234] Failure to initialize cublas may be due to OOM (cublas needs some free memory when you initialize it, and your deep-learning framework may have preallocated more than its fair share), or may be because this binary was not built with support for the GPU in your machine.
2022-06-16 15:50:16.242765: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at matmul_op_impl.h:438 : INTERNAL: Attempting to perform BLAS operation using StreamExecutor without BLAS support
2022-06-16 15:50:16.246341: E tensorflow/stream_executor/cuda/cuda_blas.cc:232] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2022-06-16 15:50:16.246367: E tensorflow/stream_executor/cuda/cuda_blas.cc:234] Failure to initialize cublas may be due to OOM (cublas needs some free memory

InternalError: Graph execution error:

Detected at node 'autoencoder_clone/sym_encoder//dense/Tensordot/MatMul' defined at (most recent call last):
    File "/math/home/eruisun/software/anaconda/envs/MRA/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/math/home/eruisun/software/anaconda/envs/MRA/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/math/home/eruisun/software/anaconda/envs/MRA/lib/python3.9/site-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/math/home/eruisun/software/anaconda/envs/MRA/lib/python3.9/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/math/home/eruisun/software/anaconda/envs/MRA/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 677, in start
      self.io_loop.start()
    File "/math/home/eruisun/software/anaconda/envs/MRA/lib/python3.9/site-